In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import types

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

23/03/06 21:53:21 WARN Utils: Your hostname, padilha-A70-HYB resolves to a loopback address: 127.0.1.1; using 192.168.15.4 instead (on interface wlo1)
23/03/06 21:53:21 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/03/06 21:53:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [3]:
df_green.registerTempTable('green')

/home/padilha/miniconda3/envs/de-zoomcamp-week5/lib/python3.9/site-packages/pyspark/sql/dataframe.py:138: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn(


In [4]:
df_green_revenue = spark.sql("""
SELECT 
    -- Reveneue grouping 
    PULocationID AS revenue_zone,
    date_trunc('hour', lpep_pickup_datetime) AS hour_month, 

    SUM(total_amount) AS revenue_monthly_total_amount,
    count(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
ORDER BY
    1, 2
""")

In [5]:
df_green_revenue.show()

+------------+-------------------+----------------------------+--------------+
|revenue_zone|         hour_month|revenue_monthly_total_amount|number_records|
+------------+-------------------+----------------------------+--------------+
|           1|2020-01-01 03:00:00|                       155.3|             1|
|           1|2020-01-03 05:00:00|                      117.39|             1|
|           1|2020-01-26 10:00:00|                       85.56|             1|
|           1|2020-01-29 18:00:00|                      108.36|             1|
|           1|2020-02-01 06:00:00|                      115.56|             1|
|           1|2020-02-06 07:00:00|                        98.3|             1|
|           1|2020-02-16 07:00:00|                        95.3|             1|
|           1|2020-02-17 13:00:00|                        82.3|             1|
|           1|2020-02-26 17:00:00|                       49.38|             1|
|           1|2020-03-02 15:00:00|                  

In [6]:
df_green_revenue \
    .repartition(20) \
    .write \
    .parquet('data/report/revenue/green', mode='overwrite')

23/03/06 21:53:28 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 95,00% for 8 writers
23/03/06 21:53:28 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 84,44% for 9 writers
23/03/06 21:53:28 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 76,00% for 10 writers
23/03/06 21:53:28 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 69,09% for 11 writers
23/03/06 21:53:28 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 63,33% for 12 writers
23/03/06 21:53:28 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 58,46% for 13 writers
23/03/06 21:53:28 WARN MemoryManager: Total allocation exceeds 95,

In [7]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')
df_yellow.registerTempTable('yellow')

In [8]:
df_yellow_revenue = spark.sql("""
SELECT 
    -- Reveneue grouping 
    PULocationID AS revenue_zone,
    date_trunc('hour', lpep_pickup_datetime) AS hour_month, 

    SUM(total_amount) AS revenue_monthly_total_amount,
    count(1) AS number_records
FROM
    yellow
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
ORDER BY
    1, 2
""")

In [9]:
df_yellow_revenue.show()

+------------+-------------------+----------------------------+--------------+
|revenue_zone|         hour_month|revenue_monthly_total_amount|number_records|
+------------+-------------------+----------------------------+--------------+
|        null|2020-01-01 00:00:00|          2215.6499999999996|            56|
|        null|2020-01-01 01:00:00|          3352.9300000000003|            83|
|        null|2020-01-01 02:00:00|          2631.2999999999997|            69|
|        null|2020-01-01 03:00:00|                     1082.22|            23|
|        null|2020-01-01 04:00:00|           694.6700000000001|            13|
|        null|2020-01-01 05:00:00|                      267.58|             5|
|        null|2020-01-01 06:00:00|                      665.66|            13|
|        null|2020-01-01 07:00:00|                     1505.83|            35|
|        null|2020-01-01 08:00:00|                     1087.18|            28|
|        null|2020-01-01 09:00:00|          1001.080

In [10]:
df_yellow_revenue.write.parquet('data/report/revenue/yellow', mode='overwrite')

In [11]:
df_yellow_revenue \
    .repartition(20) \
    .write \
    .parquet('data/report/revenue/yellow', mode='overwrite')

23/03/06 21:53:33 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 95,00% for 8 writers
23/03/06 21:53:33 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 84,44% for 9 writers
23/03/06 21:53:33 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 95,00% for 8 writers
